In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Question 1

#### Defining Basic Functions

In [2]:
GOAL_STATE = (3, 13)    

def init_env() -> np.ndarray:
    """
    Function to initialize the environment matrix
    """
    # Environment matrix
    # regular = 0    wall = 1    oil = 2     bump = 3    start = 4    end = 5
    Env = np.zeros([20, 20])

    # Defining border walls
    Env[:, [0]] = np.ones([20, 1])
    Env[:, [19]] = np.ones([20, 1])
    Env[0, :] = np.ones([1, 20])
    Env[19, :] = np.ones([1, 20])

    # Defining cell properties inside the maze
    # wall = 1
    Env[2, 5] = 1
    Env[3, 5] = 1
    Env[4, 3:17] = np.ones([1, 14])
    Env[5, 3] = 1
    Env[6, 3] = 1; Env[6, 6] = 1; Env[6, 9] = 1; Env[6, 15] = 1
    Env[7, 3] = 1; Env[7, 6] = 1; Env[7, 9] = 1; Env[7, 12:16] = np.ones([1, 4])
    Env[8, 6] = 1; Env[8, 9] = 1; Env[8, 15] = 1
    Env[9, 6] = 1; Env[9, 9] = 1; Env[9, 15] = 1
    Env[10, 1:5] = np.ones([1, 4]); Env[10, 6] = 1; Env[10, 9:11] = [1, 1]; Env[10, 15] = 1
    Env[11, 6] = 1; Env[11, 10] = 1; Env[11, 13] = 1; Env[11, 15:18] = np.ones([1, 3])
    Env[12, 3:8] = np.ones([1, 5]); Env[12, 10] = 1; Env[12, 13] = 1; Env[12, 17] = 1
    Env[13, 7] = 1; Env[13, 10] = 1; Env[13, 13] = 1; Env[13, 17] = 1
    Env[14, 7] = 1; Env[14, 10] = 1; Env[14, 13] = 1
    Env[15, 7] = 1; Env[15, 13:17] = [1, 1, 1, 1]
    Env[17, 1:3] = [1, 1]; Env[17, 7:13] = [1, 1, 1, 1, 1, 1]

    # oil = 2
    Env[2, 8] = 2; Env[2, 16] = 2
    Env[4, 2] = 2
    Env[5, 6] = 2
    Env[10, 18] = 2
    Env[14, 14] = 2
    Env[15, 10] = 2
    Env[16, 10] = 2
    Env[17, 14] = 2; Env[17, 17] = 2
    Env[18, 7] = 2

    # bump = 3
    Env[1, 11] = 3; Env[1, 12] = 3
    Env[2, 1:4] = [3, 3, 3]
    Env[5, 1] = 3; Env[5, 9] = 3; Env[5, 17] = 3
    Env[6, 17] = 3
    Env[7, 2] = 3; Env[7, 10:12] = [3, 3]; Env[7, 17] = 3
    Env[8, 17] = 3
    Env[12, 11:13] = [3, 3]
    Env[14, 1:3] = [3, 3]
    Env[15, 17:19] = [3, 3]
    Env[16, 7] = 3

    # start = 4
    Env[15, 4] = 4

    # goal = 5
    Env[GOAL_STATE[0], GOAL_STATE[1]] = 5

    return Env

def plot_env(Env, title, annot_matrix=False, show=True) -> None:
    """
    Function to plot the environment matrix
    """
    # Define colors
    colors = {
        0: [1, 1, 1],        # White
        1: [0, 0, 0],        # Black
        2: [0.55, 0, 0],     # Light Brown
        3: [0.96, 0.8, 0.6], # Dark Red
        4: [0, 0, 1],        # Green
        5: [0, 1, 0]         # Blue
    }


    rgb_maze = np.zeros((Env.shape[0], Env.shape[1], 3))
    for i in range(Env.shape[0]):
        for j in range(Env.shape[1]):
            rgb_maze[i, j] = colors.get(Env[i, j], [1, 1, 1])

    if annot_matrix is not False:
        annot_matrix = np.where(Env == 1, '', annot_matrix)

    plt.figure(figsize=(15, 10))
    sns.heatmap(Env,fmt="",  cmap=sns.color_palette([colors[i] for i in range(6)]), cbar=False,annot=annot_matrix, linewidths=0.5, linecolor='black')
    plt.axis('off')
    plt.title(title)

    if show:
        plt.show()

def plot_policy(Env, policy, title) -> None:
    """
    Function to plot the optimal policy matrix
    """
    plot_env(Env, title, annot_matrix=False, show=False)
    for i in range(20):
        for j in range(20):
            if policy[i, j] == "up":
                plt.arrow(j+0.5, i+0.85, 0, -0.5, width=0.04, color='black')  # Up
            if policy[i, j] == "right":
                plt.arrow(j+0.15, i+0.5, 0.5, 0, width=0.04, color='black')  # Right
            if policy[i, j] == "down":
                plt.arrow(j+0.5, i+0.15, 0, 0.50, width=0.04, color='black')  # Down
            if policy[i, j] == "left":
                plt.arrow(j+0.85, i+0.5, -0.5, 0, width=0.04, color='black')  # Left

    plt.show()

def plot_optimal_path(Env, path, title) -> None:
    """
    Function to plot the optimal path
    """
    plot_env(Env, title, annot_matrix=False, show=False)
    
    for state_cr, direction in path:
        r = state_cr[0] # x_coordinate
        c = state_cr[1] # y_coordinate

        if direction == 'right':
            plt.arrow(c + 0.5, r + 0.5, 0.8, 0, width=0.04, color='black')   # Right
        if direction == 'left':
            plt.arrow(c + 0.5, r + 0.5, -0.8, 0, width=0.04, color='black')  # Left
        if direction == 'up':
            plt.arrow(c + 0.5, r + 0.5, 0, -0.8, width=0.04, color='black')  # Up
        if direction == 'down':
            plt.arrow(c + 0.5, r + 0.5, 0, 0.8, width=0.04, color='black')  # Down

    plt.show()

def get_reward(next_state_type, wall_hit) -> float:
    """
    Function to return the reward of an action based on next state and if wall was hit
    """
    next_state_type = int(next_state_type)
    reward = -1 # For taking an action

    if wall_hit:         # Wall
        reward += -0.8

    if next_state_type == 2:  # Oil
        reward += -5
    elif next_state_type == 3:  # Bump
        reward += -10
    elif next_state_type == 5:  # Goal
        reward += 300
    
    return reward

def get_next_state(Env, state, action) -> tuple:
    """
    Function to return the next state given the current state and action
    """
    i, j = state[0], state[1]
    wall_hit = False

    if action == 'up':
        next_i, next_j = i - 1, j
    elif action == 'down':
        next_i, next_j = i + 1, j
    elif action == 'left':
        next_i, next_j = i, j - 1
    elif action == 'right':
        next_i, next_j = i, j + 1

    if Env[next_i, next_j] == 1:  # Wall
        next_i, next_j = i, j
        wall_hit = True
    
    return next_i, next_j, wall_hit

def get_probabilities(action, p) -> list:
    """
    Function to return the probabilities of each action given the current action
    """
    probabilities = {"up": 0, "down": 0, "left": 0, "right": 0}
    if action == "up":
        probabilities["up"] = 1 - p
        probabilities["down"] = 0
        probabilities["left"] = p / 2
        probabilities["right"] = p / 2
    elif action == "down":
        probabilities["up"] = 0
        probabilities["down"] = 1 - p
        probabilities["left"] = p / 2
        probabilities["right"] = p / 2
    elif action == "left":
        probabilities["up"] = p / 2
        probabilities["down"] = p / 2
        probabilities["left"] = 1 - p
        probabilities["right"] = 0
    elif action == "right":
        probabilities["up"] = p / 2
        probabilities["down"] = p / 2
        probabilities["left"] = 0
        probabilities["right"] = 1 - p
    
    return probabilities

def get_optimal_path(Env, start, end, optimal_policy):
    """
    Function to get the optimal path from start to end
    """
    curr_state = start
    path = []
    visited_states = []
    while curr_state != end:
        if curr_state in visited_states:
            print("No optimal path found.")
            return path

        visited_states.append(curr_state)
        i, j = curr_state[0], curr_state[1]
        action = optimal_policy[i, j]
        path.append((curr_state, action))
        next_i, next_j, wall_hit = get_next_state(Env, curr_state, action)
        curr_state = (next_i, next_j)

    return path

def calculate_state_value(Env, curr_state, action, p, V, gamma):
    """
    Function to calculate the value of a state given the next state and reward
    """
    i, j = curr_state[0], curr_state[1]
    probabilites = get_probabilities(action, p)

    possible_actions = ['up', 'down', 'left', 'right']
    state_value = 0

    for curr_action in possible_actions:
        curr_action_prob = probabilites[curr_action]
        
        next_i, next_j, wall_hit = get_next_state(Env, (i, j), curr_action)
        reward = get_reward(Env[next_i, next_j], wall_hit)

        state_value += curr_action_prob * (reward + (gamma * V[next_i, next_j]))

    return state_value

#### Policy Iteration Functions

In [ ]:
def policy_evaluation(Env, policy, p, gamma, theta):
    """
    Function to evaluate the policy using approximate policy evaluation
    """
    V = np.zeros(Env.shape)

    while True:
        delta = 0
        V_new = np.copy(V)

        for i in range(Env.shape[0]):
            for j in range(Env.shape[1]):
                if Env[i, j] == 1:  # Wall
                    continue

                if (i, j) == GOAL_STATE:  # Force goal state value to 0
                    V_new[i, j] = 0
                    continue

                curr_v = V[i, j]
                action = policy[i, j]
                curr_state = (i, j)

                V_new[i, j] = calculate_state_value(Env, curr_state, action, p, V, gamma)
                delta = max(delta, abs(curr_v - V_new[i, j]))

        V = V_new
        if delta < theta:
            break

    return V

def policy_improvement(Env, V, p, gamma):
    """
    Function to improve the policy
    """
    policy = np.full((Env.shape[0], Env.shape[1]), '', dtype=object)

    for i in range(Env.shape[0]):
        for j in range(Env.shape[1]):
            if Env[i, j] == 1:  # Wall
                continue

            curr_state = (i, j)
            possible_actions = ['up', 'down', 'left', 'right']
            action_values = []

            # Calculate value of each action
            for action in possible_actions:
                v = calculate_state_value(Env, curr_state, action, p, V, gamma)
                action_values.append(v)

            # Get best action and update policy
            best_action = possible_actions[np.argmax(action_values)]
            policy[i, j] = best_action

    return policy

def policy_iteration(Env, policy, p, gamma, theta):
    """
    Function to perform policy iteration
    """
    curr_policy = policy.copy()
    iteration = 0
    while True:
        iteration += 1

        V = policy_evaluation(Env, curr_policy, p, gamma, theta)
        updated_policy = policy_improvement(Env, V, p, gamma)

        if np.array_equal(updated_policy, curr_policy):
            break

        curr_policy = updated_policy.copy()

    return curr_policy, V, iteration

#### Value Iteration Functions

In [ ]:
def value_iteration(Env, p, gamma, theta):
    V = np.zeros(Env.shape)
    iterations = 0

    while True:
        iterations += 1
        delta = 0
        V_new = np.copy(V)

        for i in range(Env.shape[0]):
            for j in range(Env.shape[1]):
                if Env[i, j] == 1:  # Skip walls
                    continue

                if (i, j) == GOAL_STATE:  # Force goal state value to 0
                    V_new[i, j] = 0
                    continue

                curr_state = (i, j)
                curr_v = V[i, j]
                possible_actions = ['up', 'down', 'left', 'right']
                action_values = []

                # Calculate value of each action
                for action in possible_actions:
                    v = calculate_state_value(Env, curr_state, action, p, V, gamma)
                    action_values.append(v)
                V_new[i, j] = max(action_values)

                delta = max(delta, abs(V_new[i, j] - curr_v))

        V = V_new

        if delta < theta:
            break

    # Compute final policy
    policy = policy_improvement(Env, V, p, gamma)

    return policy, V, iterations

In [ ]:
def simulate(policy_method, probabilities, gammas, theta):
    Env = init_env()
    for i in range(len(probabilities)):
        p = probabilities[i]
        gamma = gammas[i]

        if policy_method == "policy iteration":
            # Policy Initialization
            policy = np.full((Env.shape[0], Env.shape[1]), 'left', dtype=object)

            # Policy Iteration
            optimal_policy, V, iteration = policy_iteration(Env, policy, p, gamma, theta)
        
        elif policy_method == "value iteration":
            optimal_policy, V, iteration = value_iteration(Env, p, gamma, theta)

        print(f"\n{policy_method}")
        print(f"Number of Iterations for p={p} and gamma={gamma}: {iteration}")
        # Plotting the optimal policy
        plot_policy(Env=Env, title=f"Optimal Policy for p={p} and gamma={gamma}", policy=optimal_policy)


        # Plotting the state values
        plot_env(Env, title=f"State Values for p={p} and gamma={gamma}", annot_matrix=np.round(V, 2))

        # Plotting the optimal path
        start = (15, 4)
        end = (3, 13)
        optimal_path = get_optimal_path(Env, start, end, optimal_policy)
        plot_optimal_path(Env=Env, title=f"Optimal Path for p={p} and gamma={gamma}", path=optimal_path)

In [ ]:
probabilities = [0.025, 0.4, 0.025]
gammas = [0.99, 0.99, 0.5]
theta = 0.01

simulate("policy iteration", probabilities, gammas, theta)

In [ ]:
simulate("value iteration", probabilities, gammas, theta)

### Question 2

In [ ]:
# Define parameters
NUM_STATES = 16
NUM_ACTIONS = 5
NUM_GENES = 4
ACTIONS = np.vstack(([0, 0, 0, 0], np.eye(NUM_GENES, dtype=int)))
C = np.array([[0, 0, -1, 0],
              [1, 0, -1, -1],
              [0, 1, 0, 0],
              [-1, 1, 1, 0]])

def get_policy_with_action(action):
    """Returns a policy with all actions as given action."""
    return np.full(NUM_STATES, action, dtype=int)

def xor_mod2(vec1, vec2):
    """Performs component-wise XOR (mod 2 addition)."""
    return (vec1 + vec2) % 2

def get_state_from_idx(idx):
    """Converts an index to a binary state."""
    return np.array(list(map(int, format(idx, '04b'))))

def v_bar(vec):
    output = np.zeros(len(vec))
    for i, elem in enumerate(vec):
        if elem > 0:
            output[i] = 1
        else:
            output[i] = 0
    
    return output

def abs_sum(vec):
    return np.sum(np.abs(vec))

def compute_N(p):
    """Returns 1 with probability p and 0 with probability 1-p."""
    N = []
    for _ in range(NUM_GENES):
        N.append(int(np.random.rand() < p))
    return N

def compute_next_state(p, state, action):
    """Computes the transition probability matrices for a given action and state."""
    N = compute_N(p)
    S_NEW = xor_mod2(xor_mod2(v_bar(C @ state), action), N)

    return S_NEW

def compute_transition_matrix(p, action):
    """Computes the transition probability matrices for a given action."""
    M = np.zeros((NUM_STATES, NUM_STATES))
    for i in range(NUM_STATES):
        si = get_state_from_idx(i)
        Csi = v_bar(C @ si)
        Csia = xor_mod2(Csi, action)

        for j in range(NUM_STATES):
            sj = get_state_from_idx(j)
            exp = abs_sum(sj - Csia)
            M[i, j] = (p ** exp) * ((1 - p) ** (4 - exp))
    
    return M

def compute_reward_function(action):
    """Defines the reward function."""
    R = np.zeros((NUM_STATES, NUM_STATES))
    for i in range(NUM_STATES):
        for j in range(NUM_STATES):
            next_state = get_state_from_idx(j)
            R[i, j] = 5 * sum(next_state) - abs_sum(action)
    return R

def init_P_R(p):
    P = np.zeros((NUM_ACTIONS, NUM_STATES, NUM_STATES))
    for a in range(NUM_ACTIONS):
        P[a] = compute_transition_matrix(p, ACTIONS[a])

    R = np.zeros((NUM_ACTIONS, NUM_STATES, NUM_STATES))
    for a in range(NUM_ACTIONS):
        R[a] = compute_reward_function(ACTIONS[a])
    
    return P, R

def compute_average_activation(policy, P, num_episodes, episode_length):
    """Computes the average activation rate over multiple episodes."""
    avg_activation = 0
    for _ in range(num_episodes):
        activation_sum = 0
        curr_state_idx = np.random.choice(NUM_STATES)
        for _ in range(episode_length):
            state = get_state_from_idx(curr_state_idx)
            activation_sum += np.sum(state)
            action = policy[curr_state_idx]
            transition_probs = P[action][curr_state_idx]
            next_state_idx = np.random.choice(NUM_STATES, p=transition_probs)
            curr_state_idx = next_state_idx
        avg_activation += activation_sum / episode_length
    avg_activation_rate = avg_activation / num_episodes
    return avg_activation_rate

def get_printable_policy(policy):
    """Prints the optimal policy."""
    output_policy = []
    for action in policy:
        output_policy.append(f"a{action+1}")
    
    return output_policy

def matrix_form_value_iteration(P, R, gamma, theta):
    """Performs Value Iteration in matrix form to compute the optimal policy."""
    V = np.zeros(NUM_STATES)
    policy = np.zeros(NUM_STATES, dtype=int)
    iterations = 0

    while True:
        delta = 0
        iterations += 1
        for s in range(NUM_STATES):
            v = V[s]
            action_values = [np.sum(P[a, s, :] * (R[a, s, :] + (gamma * V))) for a in range(NUM_ACTIONS)]
            V[s] = max(action_values)
            policy[s] = np.argmax(action_values)
            delta = max(delta, abs(v - V[s]))
        if delta < theta:
            break
    return policy, V, iterations

def matrix_form_policy_iteration(P, R, initial_policy, gamma):
    """Performs Policy Iteration in matrix form to compute the optimal policy."""
    V = np.zeros(NUM_STATES)
    policy = initial_policy
    iterations = 0

    while True:
        iterations += 1
        V = np.zeros(NUM_STATES)
        policy_stable = True
        for s in range(NUM_STATES):
            action = policy[s]
            V[s] = np.sum(P[action, s, :] * (R[action, s, :] + (gamma * V)))
        
        for s in range(NUM_STATES):
            old_action = policy[s]
            action_values = [np.sum(P[a, s, :] * (R[a, s, :] + (gamma * V))) for a in range(NUM_ACTIONS)]
            policy[s] = np.argmax(action_values)
            if old_action != policy[s]:
                policy_stable = False
        
        if policy_stable:
            break
    
    return policy, V, iterations


In [ ]:
num_episodes=100
episode_length=200
gamma = 0.95
theta = 0.01
p_values = [0.04, 0.15, 0.48]
no_control_policy = get_policy_with_action(0)

for p in p_values:
    P, R = init_P_R(p)
    policy, V, iterations = matrix_form_value_iteration(P, R, gamma, theta)
    print(f"p = {p}")
    print(f"Optimal policy: {get_printable_policy(policy)}")
    print(f"Optimal value function: {np.round(V, 4)}")
    print(f"Number of iterations: {iterations}")

    avg_activation = compute_average_activation(policy, P, num_episodes, episode_length)
    avg_activation_ncp = compute_average_activation(no_control_policy, P, num_episodes, episode_length)
    print(f"Average activation rate: {avg_activation}")
    print(f"Average activation rate with no control policy: {avg_activation_ncp}")
    print("---------------------------------------------------------\n")


print("\nPolicy Iteration")
print("p = 0.05")
P, R = init_P_R(0.05)
policy, V, iterations = matrix_form_policy_iteration(P, R, no_control_policy, gamma)
print(f"Optimal policy: {get_printable_policy(policy)}")
print(f"Optimal value function: {np.round(V, 4)}")
print(f"Number of iterations: {iterations}")

avg_activation = compute_average_activation(policy, P, num_episodes, episode_length)
avg_activation_ncp = compute_average_activation(no_control_policy, P, num_episodes, episode_length)
print(f"Average activation rate: {avg_activation}")
print(f"Average activation rate with no control policy: {avg_activation_ncp}")